### Env Config

In [1]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [2]:
# Aproximação da quantidade de linhas esperadas no arquivo final 
TOTAL_ROW_COUNT = 500000
# Randomizar amostras coletadas de cada arquivo
DO_RANDOMIZE_SAMPLE_ROWS = True

# Exportar os arquivos tratados ou não
DO_EXPORT_FILES = True

# Rodar ou não a visualização dos recursos (Demora muito mais tempo)
RUN_RESOURCE_VISUALIZATION = False

In [3]:
def export_to_file(df: DataFrame, path: str):
	if DO_EXPORT_FILES:
		if os.path.exists(f'{path}/df.csv'):
			os.remove(f'{path}/df.csv')
		if os.path.exists(f'{path}/df.parquet'):
			try:
				os.remove(f'{path}/df.parquet')
			except:
				os.system(f'rm -rf {path}/df.parquet')
		df.to_csv(f'{path}/df.csv', index=False)
		df.to_parquet(f'{path}/df.parquet', index=False)

### Criando dataframe

In [4]:
files = os.listdir('_data/raw/operacoes-credito')
files.sort()

sample_size = TOTAL_ROW_COUNT // len(files)

arrow_tables = []
for file in files:
	print(f'Processando o arquivo {file}')

	temp_df = pd.read_parquet(f'_data/raw/operacoes-credito/{file}', engine='pyarrow').sample(n=sample_size, random_state=random.randint(1, 1000))
	table = pa.Table.from_pandas(temp_df)
	arrow_tables.append(table)

combined_table = pa.concat_tables(arrow_tables)

df = combined_table.to_pandas()

Processando o arquivo 201206.parquet
Processando o arquivo 201207.parquet
Processando o arquivo 201208.parquet
Processando o arquivo 201209.parquet
Processando o arquivo 201210.parquet
Processando o arquivo 201211.parquet
Processando o arquivo 201212.parquet
Processando o arquivo 201301.parquet
Processando o arquivo 201302.parquet
Processando o arquivo 201303.parquet
Processando o arquivo 201304.parquet
Processando o arquivo 201305.parquet
Processando o arquivo 201306.parquet
Processando o arquivo 201307.parquet
Processando o arquivo 201308.parquet
Processando o arquivo 201309.parquet
Processando o arquivo 201310.parquet
Processando o arquivo 201311.parquet
Processando o arquivo 201312.parquet
Processando o arquivo 201401.parquet
Processando o arquivo 201402.parquet
Processando o arquivo 201403.parquet
Processando o arquivo 201404.parquet
Processando o arquivo 201405.parquet
Processando o arquivo 201406.parquet
Processando o arquivo 201407.parquet
Processando o arquivo 201408.parquet
P

### Ajuste de tipagem dos dados

In [5]:
df.dtypes

data_base                         object
uf                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
porte                             object
modalidade                        object
numero_de_operacoes               object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
indexador_modalidade              object
dtype: object

In [6]:
df['cliente'].value_counts()

cliente
PJ    429816
PF     70088
Name: count, dtype: int64

In [7]:
df = df.rename(columns={
	'cliente': 'classificacao',
	'cnae_secao': 'cnae',
})

In [8]:
if RUN_RESOURCE_VISUALIZATION:
	print(df['porte'].value_counts())
	print(df['modalidade'].value_counts())
	print(df['ocupacao'].value_counts())
	print(df['cnae'].value_counts())

In [9]:
df['porte'] = df['porte'].str.slice(5).str.strip()
df['modalidade'] = df['modalidade'].str.slice(5).str.strip()
df['ocupacao'] = df['ocupacao'].str.slice(5).str.strip()
df['cnae'] = df['cnae'].str.slice(5).str.strip()

In [10]:
MONETARY_COLS = ['carteira_ativa', 'carteira_inadimplida_arrastada', 'ativo_problematico']

CATEGORY_COLS = ['uf', 'ocupacao', 'cnae', 'porte', 'modalidade', 'indexador_modalidade', 'classificacao']

QUANTITY_COLS = ['numero_de_operacoes']

DATE_COLS = ['data_base']

for column in df.columns:
	if column in MONETARY_COLS:
		df[column] = df[column].str.replace(',', '.').astype(float)
		df = df.rename(columns={column: f'vl_{column}'})
	elif column in CATEGORY_COLS:
		df[column] = df[column].astype('category')
		df = df.rename(columns={column: f'ct_{column}'})
	elif column in QUANTITY_COLS:
		df[column] = df[column].str.replace('<= ', '').astype(int)
		df = df.rename(columns={column: f'qt_{column}'})
	elif column in DATE_COLS:
		df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
		df = df.rename(columns={column: f'dt_{column}'})

In [11]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
qt_numero_de_operacoes                        int64
vl_carteira_ativa                           float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
dtype: object

### Exportando para zona Trusted

In [12]:
export_to_file(df, '_data/trusted/operacoes-credito')

In [13]:
df['vl_carteira_problematica'] = df['vl_ativo_problematico'] - df['vl_carteira_inadimplida_arrastada']

In [14]:
df['vl_carteira_problematica'].describe()

count       499904.00
mean        231175.38
std        8622385.89
min        -151030.34
25%              0.00
50%              0.00
75%           5390.67
max     3148380232.32
Name: vl_carteira_problematica, dtype: float64

#### Valores menores do que 0?

In [15]:
df[df['vl_carteira_problematica'] < 0]['vl_carteira_problematica']

459810     -1976.39
259100    -24848.44
90552     -18354.76
296694      -578.03
229242     -1021.82
45747     -85650.38
63867     -32920.27
91742     -11823.58
232357     -3424.60
92183       -190.62
49719     -84207.14
91269     -17976.86
71348    -102697.06
71977    -144039.90
68151     -12846.52
374641    -12926.27
363982    -19475.35
361360    -18585.36
349004    -90531.43
346907   -124570.77
74929      -9136.43
524461     -9875.53
488385    -96457.99
628777    -33496.93
233028    -22067.12
644004    -16759.80
311089     -2898.41
602750    -17317.71
585142    -12935.33
202956     -4849.19
221017      -692.16
547775      -550.83
524261    -64889.38
363788    -76380.43
527997   -151030.34
Name: vl_carteira_problematica, dtype: float64

#### 0.05% da amostra está danificada, vamos capar em 0

In [16]:
df['vl_carteira_problematica'] = df['vl_carteira_problematica'].clip(lower=0)

In [17]:
df['vl_carteira_problematica'].describe()

count       499904.00
mean        231178.04
std        8622385.80
min              0.00
25%              0.00
50%              0.00
75%           5390.67
max     3148380232.32
Name: vl_carteira_problematica, dtype: float64

In [18]:
df['vl_carteira_ativa'] = df['vl_carteira_ativa'] - df['vl_carteira_problematica']
df.rename(columns={ 'vl_carteira_ativa': 'vl_carteira_saudavel' }, inplace=True)

In [19]:
df['vl_carteira_saudavel'].describe()

count        499904.00
mean        5318094.87
std       103628860.94
min               0.00
25%           16178.04
50%           95345.65
75%          520309.67
max     23377431150.84
Name: vl_carteira_saudavel, dtype: float64

In [20]:
df['vl_carteira_saudavel'] = df['vl_carteira_saudavel'] / df['qt_numero_de_operacoes']
df['vl_carteira_problematica'] = df['vl_carteira_problematica'] / df['qt_numero_de_operacoes']
df['vl_carteira_inadimplida_arrastada'] = df['vl_carteira_inadimplida_arrastada'] / df['qt_numero_de_operacoes']

In [21]:
df.drop(columns=['qt_numero_de_operacoes'], inplace=True)

In [22]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
vl_carteira_saudavel                        float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
vl_carteira_problematica                    float64
dtype: object

### Exportando para zona Refined

In [23]:
export_to_file(df, '_data/refined/operacoes-credito')